In [30]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Software Optimizations -- how to write cache friendly code?

## Revisiting the design of your data structures.

### What's a better data structure?

In [6]:
compare([do_render_code("./structure/array_of_objects.c", show=["//START","//END"]),do_render_code("structure/object_of_arrays.c", show=["//START","//END"])])

If the main workload is typically similar to 
```SELECT AVG(assignment_1) FROM table ```

In [24]:
compare([do_render_code("./structure/array_of_objects.c", show=["//START_SELECT","//END_SELECT"]),do_render_code("structure/object_of_arrays.c", show=["//START_SELECT","//END_SELECT"])])

Which one is better?

In [25]:
! cd structure; make array_of_objects; make object_of_arrays

make: 'array_of_objects' is up to date.
make: 'object_of_arrays' is up to date.


In [26]:
! cd structure; echo "array of objects"; time ./array_of_objects 28800 10000 0; echo "object of arrays"; time ./object_of_arrays 28800 10000 0

array of objects

real	0m7.414s
user	0m6.984s
sys	0m0.428s
object of arrays

real	0m2.023s
user	0m1.592s
sys	0m0.428s


Let's try a different machine now.

## Loop fusion/fission/interchance

In a very early lecture, we've learned the order of traversing loop matters a lot! It's actually the very first optimization in making your code cache friendly --

### Loop Interchange

In [10]:
compare([do_render_code("madd/madd_A.c",show=["//START","//END"]),do_render_code("madd/madd_B.c",show=["//START","//END"])])

Do you remember the code that let Jetson Nano suffer? What if we change it to the right hand side?

### Loop Fission

In [11]:
compare([do_render_code("loop/loop.c",show=["#ifdef A","#else"]), do_render_code("loop/loop.c",show=["#else","#endif"])])

In [12]:
! ssh htseng@nano-2 "cd courses/CS203/demo/memory/loop; make clean; make; echo \"Version A\"; valgrind --tool=cachegrind ./loop_A 524288 >& nano_A.perf ; grep 'D   refs\|D1' nano_A.perf; echo \"Version B\"; valgrind --tool=cachegrind ./loop_B 524288 >& nano_B.perf;  grep 'D   refs\|D1' nano_B.perf"

rm -f loop_A loop_B
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  -DA loop.c -o loop_A
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  loop.c -o loop_B
Version A
==7399== D   refs:       98,359,066  (61,383,502 rd   + 36,975,564 wr)
==7399== D1  misses:      1,641,865  ( 1,051,363 rd   +    590,502 wr)
==7399== D1  miss rate:         1.7% (       1.7%     +        1.6%  )
Version B
==7401== D   refs:       98,883,376  (61,645,657 rd   + 37,237,719 wr)
==7401== D1  misses:        724,359  (   330,466 rd   +    393,893 wr)
==7401== D1  miss rate:         0.7% (       0.5%     +        1.1%  )


What if we run it on an intel processor?

In [13]:
! cd ~/courses/CS203/demo/memory/loop; make clean; make; echo "Version A"; valgrind --tool=cachegrind ./loop_A 524288 >& intel_A.perf ; grep 'D   refs\|D1' intel_A.perf; echo "Version B"; valgrind --tool=cachegrind ./loop_B 524288 >& intel_B.perf;  grep 'D   refs\|D1' intel_B.perf

rm -f loop_A loop_B
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  -DA loop.c -o loop_A
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  loop.c -o loop_B
Version A
==3895823== D   refs:       74,769,919  (53,523,307 rd   + 21,246,612 wr)
==3895823== D1  misses:        658,581  (   264,697 rd   +    393,884 wr)
==3895823== D1  miss rate:         0.9% (       0.5%     +        1.9%  )
Version B
==3895831== D   refs:       75,294,212  (53,785,454 rd   + 21,508,758 wr)
==3895831== D1  misses:        724,117  (   330,235 rd   +    393,882 wr)
==3895831== D1  miss rate:         1.0% (       0.6%     +        1.8%  )


In [31]:
!cd ~/courses/CS203/demo/memory/loop; make clean; make; echo "version A"; time ./loop_A 2097152;  echo "version B"; time ./loop_B 2097152 

rm -f loop_A loop_B
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  -DA loop.c -o loop_A
gcc -DHAVE_LINUX_PERF_EVENT_H -g -O3  loop.c -o loop_B
version A
random in e:8.370370 0.002997 seconds

real	0m0.061s
user	0m0.048s
sys	0m0.012s
version B
random in e:8.370370 0.003646 seconds

real	0m0.062s
user	0m0.048s
sys	0m0.012s


## Case study: matrix multiplications

GEMM that computes C = A $\times$ B is the core of many AI/ML applications. The most naive implementation of GEMM takes $O(n^3)$. Assume it takes 1 second to perform GEMM on 1,024$\times$1,024$\times$1,024 matrices. How much time do you expect it would take for 2,048$\times$2,048$\times$2,048 matrices?

In [32]:
render_code("matrix_mul/mm.c", show=["//START","//END"])

// matrix_mul/mm.c:71-92 (22 lines)
//START
void mm(double **a, double **b, double **c)
{
  int i,j,k;
  for(i = 0; i < ARRAY_SIZE; i++)
  {
    for(j = 0; j < ARRAY_SIZE; j++)
    {
      for(k = 0; k < ARRAY_SIZE; k++)
      {
        c[i][j] += a[i][k]*b[k][j];
        #ifdef DUMP
          fprintf(stderr, "a[%d][%d], %p\n",i,k, &a[i][k]);
          fprintf(stderr, "b[%d][%d], %p\n",k,j, &b[k][j]);
          fprintf(stderr, "c[%d][%d], %p\n",i,j, &c[i][j]);
        #endif
      }
    }
  }
  return;
}
//END

In [33]:
! cd matrix_mul; make clean; make mm

rm -f blockmm mm blockmm_transpose cachegrind.* mm_dump
gcc -DHAVE_LINUX_PERF_EVENT_H -O3 mm.c perfstats.c -o mm 


In [35]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > mm.csv
! ssh escal "~/courses/CS203/demo/memory/matrix_mul/mm 32 >> ~/courses/CS203/demo/memory/matrix_mul/mm.csv ;~/courses/CS203/demo/memory/matrix_mul/mm 1024 >> ~/courses/CS203/demo/memory/matrix_mul/mm.csv ; ~/courses/CS203/demo/memory/matrix_mul/mm 2048 >> ~/courses/CS203/demo/memory/matrix_mul/mm.csv"
#! cs203 job memory "./matrix_mul/mm 1024 >> ./matrix_mul/mm.csv ; ./matrix_mul/mm 2048 >> ./matrix_mul/mm.csv"

In [36]:
display_df_mono(render_csv("matrix_mul/mm.csv"))

,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,313056,73293,0.234121,0.764057,0.000056,0.003454,473,136934
1,9675182466,10019668124,1.035605,0.250700,2.511931,0.237294,1019862777,4297891475
2,77397235504,119306176753,1.541479,0.294925,35.186423,0.330033,11350016409,34390582261


WOW! Compuational complexty breaks again! The GEMM performance go wild because of cache misses!

What kind of misses are we seeing?

In [ ]:
! make -C matrix_mul mm_dump; ./matrix_mul/mm_dump 256 >& mm_dump_address.csv

In [ ]:
! echo "element,address" > mm_dump_addresses_digest.csv 
! head -n 101 mm_dump_address.csv | grep "b\[" >> mm_dump_addresses_digest.csv
df = pd.read_csv("mm_dump_addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits)%S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

### Matrix tiling algorithm

Let's try to partition GEMM into smaller tiles!

In [37]:
render_code("matrix_mul/blockmm.c", show=["//START","//END"])

// matrix_mul/blockmm.c:55-73 (19 lines)
//START
void blockmm(double **a, double **b, double **c)
{
  int i,j,k, ii, jj, kk;
  for(i = 0; i < ARRAY_SIZE; i+=(ARRAY_SIZE/n))
  {
    for(j = 0; j < ARRAY_SIZE; j+=(ARRAY_SIZE/n))
    {
      for(k = 0; k < ARRAY_SIZE; k+=(ARRAY_SIZE/n))
      {        
          for(ii = i; ii < i+(ARRAY_SIZE/n); ii++)
            for(jj = j; jj < j+(ARRAY_SIZE/n); jj++)
              for(kk = k; kk < k+(ARRAY_SIZE/n); kk++)
                c[ii][jj] += a[ii][kk]*b[kk][jj];
      }
    }
  }  
}
//END

In [38]:
! cd matrix_mul/; make blockmm

gcc -O3 -DHAVE_LINUX_PERF_EVENT_H blockmm.c perfstats.c -o blockmm 


In [39]:
! time ./matrix_mul/mm 2048
! time ./matrix_mul/blockmm 2048 256

77387614733,129566519021,1.674254,0.193174,25.028828,-nan,0,0

real	0m25.074s
user	0m25.034s
sys	0m0.020s
79901980242,21380782432,0.267588,0.194992,4.169086,-nan,0,0

real	0m4.199s
user	0m4.173s
sys	0m0.024s


In [ ]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm.csv
! ssh escal "~/courses/CS203/demo/memory/matrix_mul/blockmm 32 1 >> ~/courses/CS203/demo/memory/matrix_mul/blockmm.csv ;~/courses/CS203/demo/memory/matrix_mul/blockmm 1024 256 >> ~/courses/CS203/demo/memory/matrix_mul/blockmm.csv ; ~/courses/CS203/demo/memory/matrix_mul/blockmm 2048 256 >> ~/courses/CS203/demo/memory/matrix_mul/blockmm.csv; ~/courses/CS203/demo/memory/matrix_mul/blockmm 4096 256 >> ~/courses/CS203/demo/memory/matrix_mul/blockmm.csv"

In [ ]:
display_df_mono(render_csv("matrix_mul/blockmm.csv"))

In [6]:
render_code("matrix_mul/blockmm_transpose.c", show=["//START","//END"])

// matrix_mul/blockmm_transpose.c:56-74 (19 lines)
//START
void blockmm_transpose(double **a, double **b, double **c)
{
  int i,j,k, ii, jj, kk;
  for(i = 0; i < ARRAY_SIZE; i+=(ARRAY_SIZE/n))
  {
    for(j = 0; j < ARRAY_SIZE; j+=(ARRAY_SIZE/n))
    {
      for(k = 0; k < ARRAY_SIZE; k+=(ARRAY_SIZE/n))
      {        
          for(ii = i; ii < i+(ARRAY_SIZE/n); ii++)
            for(jj = j; jj < j+(ARRAY_SIZE/n); jj++)
              for(kk = k; kk < k+(ARRAY_SIZE/n); kk++)
                c[ii][jj] += a[ii][kk]*b[jj][kk];
      }
    }
  }  
}
//END

### Matrix transpose

In [ ]:
! cd matrix_mul; make blockmm_transpose; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm_transpose.csv
! ~/courses/CS203/demo/memory/matrix_mul/blockmm_transpose 32 1 >> ~/courses/CS203/demo/memory/matrix_mul/blockmm_transpose.csv ;~/courses/CS203/demo/memory/matrix_mul/blockmm_transpose 1024 256 >> ~/courses/CS203/demo/memory/matrix_mul/blockmm_transpose.csv ; ~/courses/CS203/demo/memory/matrix_mul/blockmm_transpose 2048 256 >> ~/courses/CS203/demo/memory/matrix_mul/blockmm_transpose.csv; ~/courses/CS203/demo/memory/matrix_mul/blockmm_transpose 4096 256 >> ~/courses/CS203/demo/memory/matrix_mul/blockmm_transpose.csv

In [ ]:
display_df_mono(render_csv("matrix_mul/blockmm_transpose.csv"))

## Prefetch

x86 provide prefetch instructions. As a programmer, you may insert ```_mm_prefetch``` in x86 programs to perform software prefetch for your code. The gcc compiler also has a flag ```-fprefetch-loop-arrays``` to automatically insert software prefetch instructions.

### Using prefetch in matrix transpose code

The following example is a highly optimized matrix transpose code. In the example, we try to prefetch the next row.

In [10]:
render_code("./prefetch/transpose.cpp", lang="c++", show=["//START", "//END"])

// ./prefetch/transpose.cpp:42-63 (22 lines)
    //START
    //  Iterate each row.
    f_vector *y_iter = T;
    do{
        //  Iterate each column.
        f_vector *ptr_x = y_iter + block;
        f_vector *ptr_y = y_iter + row_size;

        do{

#ifdef ENABLE_PREFETCH
            _mm_prefetch((char*)(ptr_y + row_size),_MM_HINT_T0);
#endif
            swap_block(ptr_x,ptr_y,block);

            ptr_x += block;
            ptr_y += row_size;
        }while (ptr_y < stop_T);

        y_iter += iter_size;
    }while (y_iter < end);
    //END

Now, let's take a look of what's happening!

In [11]:
! cd prefetch; make clean; make
# ! echo "Without prefetch -- the baseline"; ssh htseng@celebi "lscpu | grep Model; cd courses/CS203/demo/memory/prefetch/; ./transpose"
! echo "Without prefetch -- the baseline"
! lscpu | grep Model
! ./prefetch/transpose

rm -f blockmm_sse blockmm blockmm_sse_prefetch transpose transpose_prefetch
g++ -msse4.1 -mavx -O3 transpose.cpp -o transpose 
g++ -msse4.1 -mavx -O3 -DENABLE_PREFETCH transpose.cpp -o transpose_prefetch 
Without prefetch -- the baseline
Model:                           183
Model name:                      13th Gen Intel(R) Core(TM) i7-13700
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.359454 seconds


In [12]:
# ! echo "With prefetch"; ssh htseng@celebi " cd courses/CS203/demo/memory/prefetch/; ./transpose_prefetch"
! echo "With prefetch"
! ./prefetch/transpose_prefetch

With prefetch
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.326084 seconds


In [13]:
#! cd prefetch; make clean; make; lscpu | grep Model
#! echo "Without prefetch -- the baseline"; cd prefetch/; ./transpose
#! echo "With prefetch"; cd prefetch/; ./transpose_prefetch
! ssh htseng@ninetales "cd /nfshome/htseng/courses/CS203/demo/memory/; make -C ./prefetch clean; make -C ./prefetch ; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; ssh htseng@ninetales  "/nfshome/htseng/courses/CS203/demo/memory/prefetch/transpose"
! echo "With prefetch";  ssh htseng@ninetales  "/nfshome/htseng/courses/CS203/demo/memory/prefetch/transpose_prefetch"

make: Entering directory '/nfshome/htseng/courses/CS203/demo/memory/prefetch'
rm -f blockmm_sse blockmm blockmm_sse_prefetch transpose transpose_prefetch
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/memory/prefetch'
make: Entering directory '/nfshome/htseng/courses/CS203/demo/memory/prefetch'
g++ -msse4.1 -mavx -O3 transpose.cpp -o transpose 
g++ -msse4.1 -mavx -O3 -DENABLE_PREFETCH transpose.cpp -o transpose_prefetch 
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/memory/prefetch'
Model:                           33
Model name:                      AMD Ryzen 7 5800X3D 8-Core Processor
Without prefetch -- the baseline
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.391221 seconds
With prefetch
bytes = 4294967296
Starting Data Transpose...   Done
Time: 0.337678 seconds



-- It doesn't work always!

### How much space does the following data structures need in physical memory?

In [ ]:
compare([do_render_code("./structure/memory_usage.c", show=["//START_1","//END_1"]),do_render_code("structure/memory_usage.c", show="main")])

In [ ]:
! cd structure; make clean; make memory_usage_A; ./memory_usage_A

Now, let's rearrange the data structure a little bit and see what's going on!

In [ ]:
compare([do_render_code("./structure/memory_usage.c", show=["//START_1","//END_1"]),do_render_code("structure/memory_usage.c", show=["START_2","END_2"])])
! cd structure; make memory_usage_B; ./memory_usage_B